# Train the Grader

In [1]:
import torch.utils.data as Data
import torch

data = torch.load(
    "Data/Beat_dataset/Beat_tensor_dataset/beat_no_head_rsz_75.pt")
x = data['x']
y = data['y']
batch_size = 64
dataset = Data.TensorDataset(x, y)
train_iter = Data.DataLoader(dataset, batch_size, shuffle=True, num_workers=0)

In [2]:
from GenMusic.models.mlp import MLP

mlp = MLP(75, [150, 50, 10], 2)

In [3]:
from torch import nn

lr = 1e-3
epoch = 1000
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=lr)
device = torch.device("cuda")
torch.cuda.empty_cache()  # 释放显存

In [ ]:
from GenMusic.models import train

model = train(train_iter, mlp, criterion,
              optimizer, epoch, device, autoPrint=0)

# Generate beat

In [1]:
import torch
from GenMusic.bin.rewarders import nngrader_rewarder, get_nngrader_rewarder
from GenMusic import ga_generator

rwd = get_nngrader_rewarder("runs/1708853660.6353629/best.pt")
rwd.set_tgt(torch.tensor([0.5, 0.5]))
ga = ga_generator(rwd)

In [2]:
ga.set_problem_params(maxormins=[1]*1, Dim=75, varTypes=[0]
                      * 75, lb=[0.3]*75, ub=[0.9]*75, lbin=[1]*75, ubin=[1]*75)
ga.set_problem_aimFunc(ga.aimFun_1)
ga.set_population_params()
ga.set_algorithm_params()

In [ ]:
ga.generate_ori()

In [ ]:
print(ga.best_tensor)